In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Embedding , Dense


In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label' , axis =1)

In [ ]:
y = df['label']
X.shape

In [ ]:
y.shape

In [ ]:
voc_size = 50000
messages = X.copy()
messages.reset_index(inplace = True)


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
nltk.download('stopwords')

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0 , len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
onehot_rep = [one_hot(word, voc_size) for word in corpus]
embedded_doc = pad_sequences(onehot_rep, padding = 'pre', maxlen = 15)

In [ ]:
features = 40
model = Sequential()
model.add(Embedding(voc_size , features , input_length = 15))
model.add(LSTM(100))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer = 'adam' ,metrics = ['accuracy'])
model.summary()

In [ ]:
import numpy as np
xf = np.array(embedded_doc)
yf = np.array(y)


In [ ]:
xf.shape,yf.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val , Y_train , Y_val= train_test_split(xf , yf , test_size = 0.25 , random_state = 21)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 10 , batch_size = 32, verbose = 1)

In [ ]:
Y_pred = model.predict_classes(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

In [ ]:
confusion_matrix(Y_val , Y_pred)


In [ ]:
accuracy_score(Y_val , Y_pred)